##### 构建模型

In [1]:
import torchvision
from mmengine.model import BaseModel
import torch.nn.functional as F

class ResNet(BaseModel):
    def __init__(self):
        super().__init__()
        self.resnet=torchvision.models.resnet50()
    def forward(self,imgs,labels,mode):
        x=self.resnet(imgs)
        if mode == 'loss':
            return {'loss':F.cross_entropy(x,labels)}
        elif mode == 'predict':
            return x,labels

##### 构建数据

In [16]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

norm_cfg=dict(mean=[0.491, 0.482, 0.447], std=[0.202, 0.199, 0.201])
train_dataloader=DataLoader(
        batch_size=32,
        shuffle=True,
        dataset=torchvision.datasets.CIFAR10(
            'data/cifar10',
            train=True,
            download=False,
            transform=transforms.Compose([
                transforms.RandomCrop(32,padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(**norm_cfg)
            ])
        )
)
val_dataloader=DataLoader(
        batch_size=32,
        shuffle=False,
        dataset=torchvision.datasets.CIFAR10(
            'data/cifar10',
            train=False,
            download=False,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(**norm_cfg)
            ])
        )
)

Files already downloaded and verified


##### 构建评测算子

In [18]:
from mmengine.evaluator import BaseMetric

class Accuracy(BaseMetric):
    def process(self,data_batch,data_samples):
        #单个进程的处理
        score,gt=data_samples
        #中间处理结果保存至self.results列表
        self.results.append({
            'batch_size':len(gt),
            'correct':(score.argmax(dim=1)==gt).sum().cpu()
        })
    def compute_metrics(self,results):
        total_batch=sum([item['batch_size'] for item in results])
        total_correct=sum([item['correct'] for item in results])
        #返回最终算子的字典，键为算子名称
        return dict(accuracy=100*total_correct/total_batch)

##### 构建执行器

In [19]:
from mmengine.runner import Runner
from torch.optim import SGD
my_runner=Runner(
    model=ResNet(),
    work_dir='./work_dir',
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    optim_wrapper=dict(optimizer=dict(type=SGD,lr=0.001,momentum=0.9)),
    train_cfg=dict(by_epoch=True,max_epochs=5,val_interval=1),
    val_cfg=dict(),
    val_evaluator=dict(type=Accuracy)
)
my_runner.train()

08/07 07:41:50 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.4 | packaged by conda-forge | (main, Mar 24 2022, 17:39:04) [GCC 10.3.0]
    CUDA available: True
    numpy_random_seed: 1894672389
    GPU 0,1: GeForce GTX 1080
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 10.2, V10.2.8
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 1.12.0+cu102
    PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191122 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.6.0 (Git Hash 52b5f107dd9cf10910aaa19cb47f3abf9b349815)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode

ResNet(
  (data_preprocessor): BaseDataPreprocessor()
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
